# CMIP6 Climatology Example

The is a simple python script to calculate a climatology for some example CMIP6 data

## Setup the Python Environment

In [ ]:
! module use /g/data3/hh5/public/modules/
! module load conda/analysis3

## Import python libraries

In [ ]:
import xarray
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
%matplotlib inline

## Define the files for calculating the climatology
The below options provide a list of the models, variables, experiments and ensembles that you want to calculate climatological fields for to use with the PMP analysis. The start and end year of the climatology that is calculated is also defined. Note that it is expected that the model output used here is monthly mean data.

In [ ]:
#In this simple script assume the institute_id, source_id and enesemble are related
institute_id = ['IPSL','CNRM-CERFACS']
source_id = ['IPSL-CM6A-LR','CNRM-CM6-1']
member_id = ['r1i1p1f1','r1i1p1f2']

experiment_id =['abrupt-4xCO2']

#The table_id is related to the variable type
table_id = ['Amon','SImon']
variable_id = ['tas','siconc']
grid_label = ['gr','gn']

year_start = 1850                     #what is the start year for the climatology to be calculated
year_end = 1999                       #what is the end year for the climatology to be calculated

## Loop through the desired options to calculate each climatology file
Here we loop through each of the options specified above and individually calcaulte the climatologies over the specified time range, saving the climatology file as a netCDF output to the current working directory. These files are necessary to then proceed to use the PMP tool as it is designed in version v1.1


In [ ]:
for i in range(len(source_id)):
    for j in range(len(experiment_id)):
        for k in range(len(variable_id)):
            #find the file path for the latest version of data
            file_system = '/g/data/oi10/replicas/'
            CMIP_dir = 'CMIP6/CMIP/%s/%s/%s/%s/%s/%s/%s'%(institute_id[i],source_id[i],experiment_id[j],member_id[i],\
                                                          table_id[k],variable_id[k],grid_label[k])
            latest_version = os.listdir('%s/%s'%(file_system,CMIP_dir))[-1]
                               
            path = '%s/%s/%s/*.nc'%(file_system,CMIP_dir,latest_version)
            print(path)
                   
            #open the dataset
            f = xarray.open_mfdataset(path,decode_times=False)
            f2 = xarray.decode_cf(f)
             
            #select the data within the time range chosen
            date_start = datetime.datetime(year_start, 1, 1, 0, 0)
            date_end = datetime.datetime(year_end + 1, 1, 1, 0, 0)
                
            #Take average in time over selected years to calculate the climatology
            climatology=f2.sel(time=slice(date_start, date_end)).groupby('time.month').mean(['time'])

            #save to a netcdf file
            climatology.to_netcdf('clim_CMIP6_%s_%s_%s_%i_%i.nc'%(source_id[i],experiment_id[j],variable_id[k],\
                                                                  year_start,year_end))
                
                